In [ ]:
import pandas as pd
import pyreadstat as st
path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2025\2025 2T\[MT08-MT13] Microeconometría II\Clases\Stata\t_8.dta"

df, meta = st.read_dta(path)
df.head(1)

In [ ]:
df.info()

NameError: name 'df' is not defined

In [37]:
from csdid.att_gt import ATTgt

# --- estimación principal ---
out = ATTgt(
    yname="lemp",
    tname="year",
    idname="countyreal",
    gname="first_treat",
    xformla="lemp ~ 1",   # incluye el outcome
    data=df,
    panel=True            # usa False si son repeated cross-sections
).fit(est_method="dr")    # 'dr' ≈ method(dripw) en Stata

attgt = out.summ_attgt()
print(attgt)

In [38]:
attgt_df = pd.DataFrame(getattr(attgt, "results", getattr(attgt, "summary2", None)))
display(attgt_df)

,group,year,att,post,se,l_se,c,u_se,sig
0,2004,2004,-0.010503,1,0.025294,-0.077524,2.649659,0.056518,
1,2004,2005,-0.070423,1,0.033063,-0.158028,2.649659,0.017182,
2,2004,2006,-0.137259,1,0.039244,-0.241242,2.649659,-0.033276,*
3,2004,2007,-0.100811,1,0.033960,-0.190793,2.649659,-0.010830,*
4,2006,2004,0.006520,0,0.024290,-0.057840,2.649659,0.070880,
5,2006,2005,-0.002751,0,0.020897,-0.058121,2.649659,0.052619,
6,2006,2006,-0.004595,1,0.019282,-0.055685,2.649659,0.046496,
7,2006,2007,-0.041224,1,0.020379,-0.095222,2.649659,0.012773,
8,2007,2004,0.030507,0,0.015907,-0.011640,2.649659,0.072654,
9,2007,2005,-0.002726,0,0.016766,-0.047149,2.649659,0.041697,


In [39]:
agg_simple = out.aggte(typec="simple")  # o type="simple"
print(agg_simple)



  ATT Std. Error  [95.0%  Conf. Int.]  
-0.04     0.0117 -0.0629       -0.017 *


---
Signif. codes: `*' confidence band does not cover 0
Control Group:  Never Treated , 
Anticipation Periods:  0
Estimation Method:  Doubly Robust




In [40]:
from csdid.att_gt import ATTgt
import pandas as pd

out = ATTgt(
    yname="lemp",
    tname="year",
    idname="countyreal",
    gname="first_treat",
    xformla="lemp ~ 1",
    data=df,
    panel=True
).fit(est_method="dr")

attgt = out.summ_attgt()
attgt_df = pd.DataFrame(attgt.results)
display(attgt_df)

print("\n.estat simple:")
print(out.aggte(typec="simple"))


,group,year,att,post,se,l_se,c,u_se,sig
0,2004,2004,-0.010503,1,0.024545,-0.075291,2.639561,0.054284,
1,2004,2005,-0.070423,1,0.031321,-0.153096,2.639561,0.012250,
2,2004,2006,-0.137259,1,0.037380,-0.235926,2.639561,-0.038591,*
3,2004,2007,-0.100811,1,0.032889,-0.187624,2.639561,-0.013998,*
4,2006,2004,0.006520,0,0.023122,-0.054513,2.639561,0.067553,
5,2006,2005,-0.002751,0,0.020157,-0.055958,2.639561,0.050456,
6,2006,2006,-0.004595,1,0.017736,-0.051409,2.639561,0.042219,
7,2006,2007,-0.041224,1,0.021011,-0.096686,2.639561,0.014237,
8,2007,2004,0.030507,0,0.015757,-0.011086,2.639561,0.072099,
9,2007,2005,-0.002726,0,0.017106,-0.047879,2.639561,0.042428,



.estat simple:


  ATT Std. Error  [95.0%  Conf. Int.]  
-0.04     0.0121 -0.0636      -0.0163 *


---
Signif. codes: `*' confidence band does not cover 0
Control Group:  Never Treated , 
Anticipation Periods:  0
Estimation Method:  Doubly Robust




In [5]:
import numpy as np
from scipy.stats import norm
from collections import defaultdict
from typing import Optional, Tuple, Union, Iterable

# --------- utilidades ----------
def _is_never(v, never_value):
    if pd.isna(v):
        return True
    if never_value is None:
        return (v == 0)
    return (v == never_value)

def _mean_or_nan(s: pd.Series) -> float:
    return float(s.mean()) if len(s) else np.nan

# --------- DIAGNÓSTICO: cuentas por (g,t) ----------
def diag_counts(df: pd.DataFrame,
                y_col="lemp", t_col="year", g_col="first_treat",
                never_value: Optional[Union[int, float]] = 0):
    d = df.copy()
    # normalizo never a 0 si corresponde
    if never_value is not None:
        d[g_col] = d[g_col].fillna(never_value)
    Gs = sorted({int(g) for g in d[g_col].dropna().unique() if g != 0})
    Ts = sorted(d[t_col].unique().tolist())

    rows = []
    for g in Gs:
        t_pre = g - 1
        if t_pre not in Ts:
            continue
        for t in Ts:
            if t < g:
                continue
            treated_t   = d[(d[g_col] == g) & (d[t_col] == t)]
            treated_pre = d[(d[g_col] == g) & (d[t_col] == t_pre)]

            ctrl_t  = d[((d[g_col] > t) | d[g_col].apply(lambda v: _is_never(v, never_value))) & (d[t_col] == t)]
            ctrl_pre= d[((d[g_col] >= g) | d[g_col].apply(lambda v: _is_never(v, never_value))) & (d[t_col] == t_pre)]

            rows.append({
                "g": g, "t": t,
                "n_treat_t": len(treated_t), "n_treat_pre": len(treated_pre),
                "n_ctrl_t": len(ctrl_t), "n_ctrl_pre": len(ctrl_pre),
                "mean_treat_t": _mean_or_nan(treated_t[y_col]),
                "mean_treat_pre": _mean_or_nan(treated_pre[y_col]),
                "mean_ctrl_t": _mean_or_nan(ctrl_t[y_col]),
                "mean_ctrl_pre": _mean_or_nan(ctrl_pre[y_col]),
            })
    out = pd.DataFrame(rows).sort_values(["g","t"])
    print(out.to_string(index=False))
    return out

# --------- ATT(g,t) sin covariables ----------
def csdid_att_gt_rc_nox(
    df: pd.DataFrame,
    y_col="lemp", t_col="year", g_col="first_treat",
    never_value: Optional[Union[int, float]] = 0,
    drop_incomplete=True,
):
    d = df.copy()
    if never_value is not None:
        d[g_col] = d[g_col].fillna(never_value)
    Gs = sorted({int(g) for g in d[g_col].dropna().unique() if g != 0})
    Ts = sorted(d[t_col].unique().tolist())

    rows = []
    for g in Gs:
        t_pre = g - 1
        if t_pre not in Ts:
            continue
        for t in Ts:
            if t < g:
                continue

            treated_t   = d[(d[g_col] == g) & (d[t_col] == t)]
            treated_pre = d[(d[g_col] == g) & (d[t_col] == t_pre)]
            ctrl_t  = d[((d[g_col] > t) | d[g_col].apply(lambda v: _is_never(v, never_value))) & (d[t_col] == t)]
            ctrl_pre= d[((d[g_col] >= g) | d[g_col].apply(lambda v: _is_never(v, never_value))) & (d[t_col] == t_pre)]

            m_tt   = _mean_or_nan(treated_t[y_col])
            m_tp   = _mean_or_nan(treated_pre[y_col])
            m_ct   = _mean_or_nan(ctrl_t[y_col])
            m_cp   = _mean_or_nan(ctrl_pre[y_col])

            if drop_incomplete and (np.isnan(m_tt) or np.isnan(m_tp) or np.isnan(m_ct) or np.isnan(m_cp)):
                continue  # evita pares mal formados

            att = (m_tt - m_tp) - (m_ct - m_cp)
            rows.append({
                "g": int(g), "t": int(t),
                "att": att,
                "n_treat_t": len(treated_t), "n_treat_pre": len(treated_pre),
                "n_ctrl_t": len(ctrl_t), "n_ctrl_pre": len(ctrl_pre),
            })
    return pd.DataFrame(rows).sort_values(["g","t"]).reset_index(drop=True)

# --------- Bootstrap (cluster opcional por unidad) ----------
def _cluster_boot_indices(ids: np.ndarray, rng: np.random.Generator):
    uniq = np.unique(ids)
    pick = rng.integers(0, len(uniq), size=len(uniq))
    chosen = uniq[pick]
    # muestrar clusters con reemplazo y concatenar sus filas
    idx_list = [np.where(ids == c)[0] for c in chosen]
    return np.concatenate(idx_list)

def csdid_bootstrap(
    df: pd.DataFrame,
    y_col="lemp", t_col="year", g_col="first_treat",
    id_col="countyreal",
    n_boot=999, seed=2025,
    cluster: bool = True,
    never_value: Optional[Union[int, float]] = 0,
):
    rng = np.random.default_rng(seed)
    base = csdid_att_gt_rc_nox(df, y_col, t_col, g_col, never_value)

    keys = list(base[["g","t"]].itertuples(index=False, name=None))
    att0 = {(g,t): v for g,t,v in base[["g","t","att"]].itertuples(index=False)}
    boots = defaultdict(list)

    ids = df[id_col].values if id_col in df.columns else None
    for _ in range(n_boot):
        if cluster and (ids is not None):
            b_idx = _cluster_boot_indices(ids, rng)
        else:
            b_idx = rng.integers(0, len(df), size=len(df))
        bdf = df.iloc[b_idx].reset_index(drop=True)
        bgt = csdid_att_gt_rc_nox(bdf, y_col, t_col, g_col, never_value)
        bmap = {(int(r.g), int(r.t)): float(r.att) for r in bgt.itertuples(index=False)}
        for k in keys:
            boots[k].append(bmap.get(k, att0[k]))

    rows = []
    for r in base.itertuples(index=False):
        g, t, att = int(r.g), int(r.t), float(r.att)
        arr = np.asarray(boots[(g,t)], dtype=float)
        se  = float(arr.std(ddof=1))
        z   = att / se if se > 0 else np.nan
        p   = 2*(1 - norm.cdf(abs(z))) if se > 0 else np.nan
        ci_lo, ci_hi = att - 1.96*se, att + 1.96*se
        rows.append({
            "g": g, "t": t,
            "g_label": f"g{g}",
            "t_label": f"t_{t-1}_{t}",
            "Coefficient": att, "Std. Err.": se, "z": z, "P>|z|": p,
            "ci_lo": ci_lo, "ci_hi": ci_hi,
            "n_treat_t": r.n_treat_t, "n_treat_pre": r.n_treat_pre,
            "n_ctrl_t": r.n_ctrl_t, "n_ctrl_pre": r.n_ctrl_pre,
        })
    gt = pd.DataFrame(rows).sort_values(["g","t"]).reset_index(drop=True)

    # estat simple (promedio simple sobre pares válidos)
    atts = gt["Coefficient"].to_numpy()
    att_s = float(np.nanmean(atts)) if len(atts) else np.nan

    boot_s = []
    for _ in range(n_boot):
        if cluster and (ids is not None):
            b_idx = _cluster_boot_indices(ids, rng)
        else:
            b_idx = rng.integers(0, len(df), size=len(df))
        bdf = df.iloc[b_idx].reset_index(drop=True)
        bgt = csdid_att_gt_rc_nox(bdf, y_col, t_col, g_col, never_value)
        boot_s.append(float(bgt["att"].mean()) if len(bgt) else np.nan)
    boot_s = np.asarray([x for x in boot_s if not np.isnan(x)])
    se_s = float(boot_s.std(ddof=1)) if len(boot_s) else np.nan
    z_s  = att_s / se_s if (se_s is not None and se_s > 0) else np.nan
    p_s  = 2*(1 - norm.cdf(abs(z_s))) if (se_s is not None and se_s > 0) else np.nan
    ci_lo_s, ci_hi_s = (att_s - 1.96*se_s, att_s + 1.96*se_s) if se_s and not np.isnan(se_s) else (np.nan, np.nan)

    simple = pd.DataFrame([{
        "ATT": att_s, "Std. Err.": se_s, "z": z_s, "P>|z|": p_s,
        "ci_lo": ci_lo_s, "ci_hi": ci_hi_s
    }])
    return gt, simple

# --------- Impresión estilo Stata ----------
def print_stata_like(gt: pd.DataFrame, simple: pd.DataFrame):
    print("Difference-in-difference with Multiple Time Periods\n")
    print("Outcome model  : least squares")
    print("Treatment model: inverse probability\n")
    print("-"*78)
    print(f"{'':13}| {'Coefficient':>11}  {'Std. err.':>9}  {'z':>7}  {'P>|z|':>6}  {'[95% conf. interval]':>23}")
    print("-"*78)
    for g, gdf in gt.groupby("g", sort=True):
        print(f"g{g:<11}|")
        for _, r in gdf.iterrows():
            print(f" {r['t_label']:<12}|  {r['Coefficient']:>11.7f}  {r['Std. Err.']:>9.6f}  "
                  f"{(r['z'] if pd.notna(r['z']) else np.nan):>7.2f}  "
                  f"{(r['P>|z|'] if pd.notna(r['P>|z|']) else np.nan):>6.3f}  "
                  f"{r['ci_lo']:>11.7f}   {r['ci_hi']:>11.7f}")
        print("-"*78)
    print("Control: Never Treated\n")
    print("See Callaway and Sant'Anna (2021) for details\n")
    print(". estat simple")
    print("Average Treatment Effect on Treated")
    print("-"*78)
    r = simple.iloc[0]
    print(f"{'ATT':>13} |  {r['ATT']:>11.7f}  {r['Std. Err.']:>9.6f}  {r['z']:>7.2f}  "
          f"{r['P>|z|']:>6.3f}  {r['ci_lo']:>11.7f}   {r['ci_hi']:>11.7f}")
    print("-"*78)

In [6]:
# Asegurate de que tu df tiene estas columnas:
# year, countyreal, lemp, first_treat
# Si "never" no es 0 ni NaN, indicalo en never_value.

# 1) Diagnóstico: revisá que los conteos no estén en cero
_ = diag_counts(df, y_col="lemp", t_col="year", g_col="first_treat", never_value=0)

# 2) Estimación con cluster bootstrap por unidad (más estable)
gt, simple = csdid_bootstrap(
    df,
    y_col="lemp", t_col="year", g_col="first_treat", id_col="countyreal",
    n_boot=999, seed=2025, cluster=True, never_value=0
)

print_stata_like(gt, simple)

   g    t  n_treat_t  n_treat_pre  n_ctrl_t  n_ctrl_pre  mean_treat_t  mean_treat_pre  mean_ctrl_t  mean_ctrl_pre
2004 2004         20           20       480         500      6.106564        6.179697     5.728867       5.798510
2004 2005         20           20       480         500      6.059452        6.179697     5.740702       5.798510
2004 2006         20           20       440         500      6.026704        6.179697     5.693967       5.798510
2004 2007         20           20       309         500      6.085388        6.179697     5.661133       5.798510
2006 2006         40           40       440         480      6.557435        6.527941     5.693967       5.740702
2006 2007         40           40       309         480      6.543041        6.527941     5.661133       5.740702
2007 2007        131          131       309         440      5.820048        5.823866     5.661133       5.693967
Difference-in-difference with Multiple Time Periods

Outcome model  : least squares
Trea